## Feature Extraction

In [1]:
import os
import librosa
import numpy as np
import pandas as pd


In [2]:
def extract_features(file_path, duration = 140):
    try:
        # Load the audio file
        audio, sr = librosa.load(file_path, sr=22050, duration = duration)
        
        # Extract features
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_fft = 441, hop_length = 220, n_mfcc=13).T, axis=0)  # MFCCs
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sr, n_fft = 441, hop_length = 220).T, axis=0)     # Chroma
        mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr, n_fft = 441, hop_length = 220).T, axis=0)     # Mel Spectrogram
        contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sr, n_fft = 441, hop_length = 220).T, axis=0)  # Spectral Contrast

        
        return np.hstack([mfccs, chroma, mel, contrast])
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [3]:
def process_audio_files(base_path, duration = 140):
    data = []
    for label in ['male', 'female']:  # Assuming folders are named 'male' and 'female'
        folder_path = os.path.join(base_path, label)
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.ogg'):  # Check for .ogg files
                file_path = os.path.join(folder_path, file_name)
                features = extract_features(file_path, duration= duration)
                if features is not None:
                    data.append([*features, label])  # Append features and label
    
    # Create a DataFrame
    columns = [f'mfcc_{i+1}' for i in range(13)] + \
              [f'chroma_{i+1}' for i in range(12)] + \
              [f'mel_{i+1}' for i in range(128)] + \
              [f'contrast_{i+1}' for i in range(7)] + \
              ['label']
    return pd.DataFrame(data, columns=columns)


In [7]:
# Set the base path
base_path = r"D:\Audios"

# Extract features and create the dataset
audio_data = process_audio_files(base_path)

# Save the dataset to a CSV file
audio_data.to_csv('gender_audio_features_Final_greeshma.csv', index=False)

print("Feature extraction completed. Dataset saved to 'gender_audio_features.csv'.")


Error processing D:\Audios\female\41.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\42.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\43.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio12.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio31.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio34.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio37.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audiof11.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audiof8.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio_173.ogg: mo

## DENOISING 

In [8]:
# Define the Band-Pass Filter function
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, data)

In [ ]:
# Feature extraction function with filtering
def extract_features(file_path, duration=140, lowcut=85.0, highcut=8000.0):
    try:
        # Load the audio file
        audio, sr = librosa.load(file_path, sr=22050, duration=duration)
        
        # Apply band-pass filter
        filtered_audio = bandpass_filter(audio, lowcut, highcut, sr)
        
        # Extract features
        mfccs = np.mean(librosa.feature.mfcc(y=filtered_audio, sr=sr, n_fft = 441, hop_length = 220, n_mfcc=13).T, axis=0)  # MFCCs
        chroma = np.mean(librosa.feature.chroma_stft(y=filtered_audio, sr=sr, n_fft = 441, hop_length = 220).T, axis=0)     # Chroma
        mel = np.mean(librosa.feature.melspectrogram(y=filtered_audio, sr=sr, n_fft = 441, hop_length = 220).T, axis=0)     # Mel Spectrogram
        contrast = np.mean(librosa.feature.spectral_contrast(y=filtered_audio, sr=sr, n_fft = 441, hop_length = 220).T, axis=0)  # Spectral Contrast
        
        return np.hstack([mfccs, chroma, mel, contrast])
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [10]:
# Function to process all audio files in the dataset
def process_audio_files(base_path, duration=140, lowcut=85.0, highcut=8000.0):
    data = []
    for label in ['male', 'female']:  # Assuming folders are named 'male' and 'female'
        folder_path = os.path.join(base_path, label)
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.ogg'):  # Check for .ogg files
                file_path = os.path.join(folder_path, file_name)
                features = extract_features(file_path, duration=duration, lowcut=lowcut, highcut=highcut)
                if features is not None:
                    data.append([*features, label])  # Append features and label
    
    # Create a DataFrame
    columns = [f'mfcc_{i+1}' for i in range(13)] + \
              [f'chroma_{i+1}' for i in range(12)] + \
              [f'mel_{i+1}' for i in range(128)] + \
              [f'contrast_{i+1}' for i in range(7)] + \
              ['label']
    return pd.DataFrame(data, columns=columns)

In [11]:
from scipy.signal import butter, lfilter

In [12]:
# Set the base path
base_path = r"D:\Audios"

# Extract features and create the dataset
audio_data = process_audio_files(base_path, duration=140)

# Save the dataset to a CSV file
audio_data.to_csv('gender_audio_features_denoised_greeshma.csv', index=False)

print("Feature extraction with denoising completed. Dataset saved to 'gender_audio_features_denoised.csv'.")

Error processing D:\Audios\female\41.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\42.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\43.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio12.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio31.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio34.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio37.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audiof11.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audiof8.ogg: module 'soundfile' has no attribute 'SoundFileRuntimeError'
Error processing D:\Audios\female\audio_173.ogg: mo

In [14]:
import pandas as pd
from scipy.stats import ttest_ind
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the datasets (replace with the correct file paths)
original_dataset = pd.read_csv('gender_audio_features_Final_greeshma.csv')  # Path to original dataset
denoised_dataset = pd.read_csv('gender_audio_features_denoised_greeshma.csv')  # Path to denoised dataset

# Function to preprocess the dataset
def preprocess_dataset(dataset):
    # Separate features and labels
    X = dataset.iloc[:, :-1]  # All columns except the last are features
    y = dataset.iloc[:, -1]   # The last column is the label

    # Encode labels (if not already encoded)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    # Normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Convert normalized X back to a DataFrame for easier handling
    X = pd.DataFrame(X, columns=dataset.columns[:-1])

    return X, y

# Preprocess both datasets
X_original, y_original = preprocess_dataset(original_dataset)
X_denoised, y_denoised = preprocess_dataset(denoised_dataset)

# Function to perform t-tests on features
def perform_t_tests(X, y):
    # Separate data by class (assuming 0 for Female, 1 for Male)
    group_0 = X[y == 0]  # Features for Female
    group_1 = X[y == 1]  # Features for Male

    # Perform t-test for each feature
    t_test_results = []
    for feature in range(X.shape[1]):
        t_stat, p_value = ttest_ind(group_0.iloc[:, feature], group_1.iloc[:, feature], equal_var=False)
        t_test_results.append({'Feature': X.columns[feature], 't-stat': t_stat, 'p-value': p_value})

    return pd.DataFrame(t_test_results)

# Perform t-tests on both datasets
t_test_results_original = perform_t_tests(X_original, y_original)
t_test_results_denoised = perform_t_tests(X_denoised, y_denoised)

# Save results to CSV for analysis
t_test_results_original.to_csv('t_test_results_original.csv', index=False)
t_test_results_denoised.to_csv('t_test_results_denoised.csv', index=False)

# Print results for quick view
print("T-Test Results for Original Dataset:")
print(t_test_results_original)

print("\nT-Test Results for Denoised Dataset:")
print(t_test_results_denoised)


T-Test Results for Original Dataset:
        Feature    t-stat       p-value
0        mfcc_1 -0.895155  3.718334e-01
1        mfcc_2 -3.507170  5.898417e-04
2        mfcc_3 -0.221295  8.251007e-01
3        mfcc_4 -5.674025  6.050289e-08
4        mfcc_5 -5.101659  9.666642e-07
..          ...       ...           ...
155  contrast_3  5.503707  1.335542e-07
156  contrast_4  2.313023  2.192591e-02
157  contrast_5  0.460882  6.455465e-01
158  contrast_6 -2.169235  3.142753e-02
159  contrast_7 -0.821837  4.121347e-01

[160 rows x 3 columns]

T-Test Results for Denoised Dataset:
        Feature    t-stat       p-value
0        mfcc_1 -0.810958  4.183983e-01
1        mfcc_2 -3.772705  2.303710e-04
2        mfcc_3 -0.678700  4.981350e-01
3        mfcc_4 -5.153316  7.654924e-07
4        mfcc_5 -6.052766  8.900398e-09
..          ...       ...           ...
155  contrast_3  5.397354  2.212651e-07
156  contrast_4  2.246234  2.597780e-02
157  contrast_5  0.428838  6.686489e-01
158  contrast_6 -2.17